In [ ]:
!pip install torch -U
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/s eta 0:00:00


In [ ]:
import transformers
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from transformers import EarlyStoppingCallback

import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

### GPU 설정하기,

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print("device : " , device)

device :  cuda:0


## 토크나이저 및 모델 불러오기

In [ ]:
Model_name = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(Model_name)

## 데이터 전처리

In [ ]:
import pandas as pd
data1 = pd.read_excel("/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/data/word_out_언감O.xlsx", index_col = 0)
data2 = pd.read_excel("/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/data/word_in_언감O.xlsx", index_col = 0)
data3 = pd.read_excel("/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/data/wrong.xlsx", index_col = 0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
data = pd.concat([data1, data2, data3], axis = 0)

In [ ]:
data.index = range(len(data))

In [ ]:
data

In [ ]:
dfa = pd.read_csv("/content/drive/MyDrive/데이터 청년 캠퍼스/사전/챗지피티 애드온 사전.csv", index_col = 0)

In [ ]:
number = pd.DataFrame()
number['word1'] = None
number['word'] = None

In [ ]:
dfa.dropna(subset = ['from'],inplace =True)

In [ ]:
s = data3['sentence']

In [ ]:
for integer ,word in enumerate(list(dfa['from'])):
  filtered_s = s[s.str.contains(word)]
  number.loc[integer, 'word1' ] = len(filtered_s)
  number.loc[integer, 'word'] = word

In [ ]:
number.sort_values(by=  'word1', ascending = False)[:15]

,word1,word
148,33301,공부
424,8010,출산
287,7220,부부
454,6256,가수
749,5661,존나
742,4129,존맛
443,4032,학부모
502,3594,서민
498,3095,비정규직
812,2851,왼쪽


In [ ]:
empty = []
empty = empty + list(index1.index)
empty = empty + list(index2.index)
empty = empty + list(index3.index)
empty = empty + list(index4.index)
empty = empty + list(index5.index)
empty = empty + list(index6.index)
empty = empty + list(index7.index)
empty = empty + list(index8.index)
empty = empty + list(index9.index)
empty = empty + list(index10.index)
empty = empty + list(index11.index)
empty = empty + list(index12.index)
empty = empty + list(index13.index)

NameError: ignored

In [ ]:
data3 = data3.drop(list(data3.loc[empty].index),axis=0)

In [ ]:
s = data3['sentence'].str.contains("공부")
index1 = data3['sentence'][s].sample(n = 33301-2700)
s = data3['sentence'].str.contains("출산")
index2 = data3['sentence'][s].sample(n = 8010- 2700)
s = data3['sentence'].str.contains("부부")
index3 = data3['sentence'][s].sample(n = 7220-2700)
s = data3['sentence'].str.contains("가수")
index4 = data3['sentence'][s].sample(n = 6256- 2700)
s = data3['sentence'].str.contains("존나")
index5 = data3['sentence'][s].sample(n = 5661-2700)
s = data3['sentence'].str.contains("존맛")
index6 = data3['sentence'][s].sample(n = 4129- 2700)
s = data3['sentence'].str.contains("학부모")
index7 = data3['sentence'][s].sample(n = 4032-2700)
s = data3['sentence'].str.contains("서민")
index8 = data3['sentence'][s].sample(n = 3594 - 2700)

s = data3['sentence'].str.contains("비정규직")
index9 = data3['sentence'][s].sample(n = 3095-2700)
s = data3['sentence'].str.contains("왼쪽")
index10 = data3['sentence'][s].sample(n = 2851- 2700)
s = data3['sentence'].str.contains("아내")
index11 = data3['sentence'][s].sample(n = 2833-2700)
s = data3['sentence'].str.contains("지랄")
index12 = data3['sentence'][s].sample(n = 2802- 2700)
s = data3['sentence'].str.contains("오른쪽")
index13 = data3['sentence'][s].sample(n = 2798- 2700)


In [ ]:
number.sort_values(by=  'word1', ascending = False)[:10]

,word1,word
148,33301,공부
424,8010,출산
287,7220,부부
454,6256,가수
749,5661,존나
742,4129,존맛
443,4032,학부모
502,3594,서민
498,3095,비정규직
812,2851,왼쪽


In [ ]:
list(dfa['from'])

['기형',
 '기형적',
 '결정장애',
 '긴팔',
 '간질',
 '간질병자',
 '귀머거리',
 '꺽다리',
 '깜깜이',
 '곱사등이',
 '곰배팔이',
 '곱추',
 '꼽추',
 '간질장애인',
 '깜깜이환자',
 '기형아',
 '농아자',
 '농아',
 '눈먼돈',
 '난쟁이',
 '난장이',
 '나병환자',
 '나병',
 '등신',
 '뗑깡',
 '땡깡',
 '땅딸보',
 '맹인',
 '문맹',
 '미친놈',
 '미친년',
 '미치광이',
 '미친사람',
 '머저리',
 '문둥이',
 '말더듬이',
 '문디',
 '문디자슥',
 '미숙아',
 '불구',
 '불구자',
 '백치',
 '바보 천치',
 '바보천치',
 '봉사',
 '벙어리',
 '반팔 티',
 '반팔티',
 '반팔',
 '백치미',
 '벙어리장갑',
 '반신불수',
 '배냇병신',
 '병신',
 '병크',
 '병맛',
 '병신크리',
 '반벙어리',
 '빙신',
 '발암캐',
 '발암케',
 '발연기',
 '발암이',
 '바보',
 '선택장애',
 '성불구',
 '성불구자',
 '소경',
 '사팔뜨기',
 '심신장애자',
 '사팔',
 '셀카고자',
 '수화',
 '외발자전거',
 '애자',
 '앉은뱅이',
 '애꾸눈',
 '애꾸눈이',
 '얼간이',
 '염병',
 '외팔이',
 '외눈박이',
 '육손이',
 '언청이',
 '언청샌님',
 '암걸리',
 '에이즈환자',
 '연애고자',
 '장애우',
 '장애자',
 '장애인놀이',
 '저능아',
 '장님',
 '조막손',
 '절뚝이',
 '쩔뚝이',
 '자폐급',
 '자폐같',
 '자폐같은',
 '자폐같다는',
 '자폐같아',
 '자폐같다',
 '자폐같아요',
 '자폐같습니다',
 '자폐같다고',
 '자폐같이',
 '자폐같게',
 '자폐같고',
 '자폐같아서',
 '자폐같으니',
 '자폐같지만',
 '정병급',
 '정신지체',
 '정신분열증',
 '정신박약아',
 '정박아',
 '작다리',
 '절름발이',
 '절름발이정책',
 '정신병

In [ ]:
s = data3['sentence']

# 조건에 해당하지 않는 문장만 선택
filtered_s = s[~s.str.contains(pattern, case=False)]

NameError: ignored

In [ ]:
filtered= s[s.str.contains(pattern, case=False)]

AttributeError: ignored

In [ ]:
data = data.loc[list(filtered_s.index)]
data.index = range(len(data))

In [ ]:
data

,sentence,장애및병력,성과가족,사회적신분,출신,욕설,기타,언어감수성_X,언어감수성_O
0,오오 갔다가 내려와서 막걸리도 한잔하자,0,0,0,0,0,0,0,1
1,그리고 학습 안못해서 생긴 스트레스는 학습해야만 풀리는듯,0,0,0,0,0,0,0,1


In [ ]:
s = data['sentence']

In [ ]:
s

0              오오 갔다가 내려와서 막걸리도 한잔하자
1    그리고 학습 안못해서 생긴 스트레스는 학습해야만 풀리는듯
Name: sentence, dtype: object

In [ ]:
data = data[(~data['sentence'].str.contains('공부'))&(~data['sentence'].str.contains('부부'))&(~data['sentence'].str.contains('출산'))&(~data['sentence'].str.contains('가수'))]

In [ ]:
data

,sentence,장애및병력,성과가족,사회적신분,출신,욕설,기타,언어감수성_X,언어감수성_O
0,오오 갔다가 내려와서 막걸리도 한잔하자,0,0,0,0,0,0,0,1
1,그리고 학습 안못해서 생긴 스트레스는 학습해야만 풀리는듯,0,0,0,0,0,0,0,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['언어감수성_X'], test_size=0.2, shuffle = True)

In [ ]:
tokenized_train_sentence = tokenizer(
    list(X_train),
    return_tensors = 'pt',
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True)

In [ ]:
print(tokenized_train_sentence[0].tokens)

['[CLS]', '전', '오늘', '몸', '##치', '##인거', '들', '##켰', '##어', '##여', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
tokenized_test_sentence = tokenizer(
    list(X_test),
    return_tensors = 'pt',
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True)

### 이젠 데이터를 torch 버전으로 변환해주기

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

### model 학습

In [ ]:
train_label = y_train.values
test_label = y_test.values

train_dataset = CurseDataset(tokenized_train_sentence, train_label)
test_dataset = CurseDataset(tokenized_test_sentence, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels = 2)
model.to(device)

In [ ]:

import accelerate

In [ ]:
training = TrainingArguments(
    output_dir='./',
    num_train_epochs = 40,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=500,
    save_total_limit=2,
    save_strategy = 'steps',
    evaluation_strategy="steps",  # 이를 "steps" 또는 "epoch"으로 설정하여 정기적으로 평가를 수행
    eval_steps=500,  # 평가를 얼마나 자주 수행할지 (500 스텝마다 평가)
    load_best_model_at_end=True,  # 학습이 끝날 때 최고의 모델을 로드
    metric_for_best_model="f1",  # 모델의 성능을 비교하기 위한 메트릭 설정 (여기서는 f1 점수)
    greater_is_better=True,  # f1 점수가 클수록 좋으므로 True로 설정
      # 개선이 없을 때 얼마나 많은 평가를 기다릴 것인지
)


In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision , recall, f1, _ = precision_recall_fscore_support(labels, preds, average = 'binary')
  acc = accuracy_score(labels, preds)
  return {
      'accuracy ' : acc,
      'f1': f1,
      'precision' : precision,
      'recall' : recall
  }

In [ ]:
trainer = Trainer(
    model = model,
    args = training,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

<ipython-input-149-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.228800,0.129964,0.970693,0.950514,0.954282,0.946776
1000,0.105100,0.074452,0.979402,0.965057,0.973498,0.956761
1500,0.080300,0.069228,0.983015,0.971235,0.978030,0.964533
2000,0.071800,0.061155,0.986502,0.977304,0.977028,0.977580
2500,0.060800,0.058156,0.986334,0.977115,0.972872,0.981395
3000,0.062000,0.059525,0.987510,0.979119,0.973286,0.985022
3500,0.051500,0.056050,0.988028,0.979973,0.974699,0.985305
4000,0.052900,0.065026,0.985185,0.974805,0.985839,0.964015
4500,0.055500,0.048975,0.988518,0.980761,0.977093,0.984457
5000,0.051300,0.041403,0.990478,0.983981,0.984259,0.983703


<ipython-input-149-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-149-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-149-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-149-d3da3f7a2d94>:7: UserWarning: To copy construct fro

TrainOutput(global_step=10000, training_loss=0.060018207263946535, metrics={'train_runtime': 11666.4487, 'train_samples_per_second': 979.434, 'train_steps_per_second': 30.607, 'total_flos': 1.874660411107908e+16, 'train_loss': 0.060018207263946535, 'epoch': 1.12})

In [ ]:
test_dataset

NameError: ignored

In [ ]:
trainer.evaluate(eval_dataset = test_dataset)

<ipython-input-149-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.02961510233581066,
 'eval_accuracy ': 0.9933208244651058,
 'eval_f1': 0.9887703933893637,
 'eval_precision': 0.9884213499011578,
 'eval_recall': 0.9891196834817013,
 'eval_runtime': 289.3242,
 'eval_samples_per_second': 246.837,
 'eval_steps_per_second': 3.857,
 'epoch': 1.12}

In [ ]:
def predict(sent):
  model.eval()

  tokenized_sent = tokenizer(
      sent,
      return_tensors = 'pt',
      truncation = True,
      add_special_tokens = True,
      max_length = 128
  )

  tokenized_sent.to(device)

  with torch.no_grad():
    outputs = model(
        input_ids = tokenized_sent['input_ids'],
        attention_mask = tokenized_sent['attention_mask'],
        token_type_ids = tokenized_sent['token_type_ids']
    )

  logits = outputs[0]
  logits = logits.detach().cpu()
  result = logits.argmax(-1)

  if result == 0:
    result = '언어감수성이 낮습니다'
  elif result == 1:
    result = '언어감수성이 높습니다'
  return result

In [ ]:
predict("너 오른손잡이야?")

NameError: ignored

In [ ]:
model.save_pretrained('directory_path')
tokenizer.save_pretrained('directory_path')

('directory_path/tokenizer_config.json',
 'directory_path/special_tokens_map.json',
 'directory_path/tokenizer.json')

In [ ]:
from google.colab import files

# 저장된 폴더를 zip 파일로 압축
!zip -r /content/directory_path.zip /content/directory_path

In [ ]:
# 모델 저장 경로 설정
save_path = "/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/Result"

# 모델과 토크나이저 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/Result/tokenizer_config.json',
 '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/Result/special_tokens_map.json',
 '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/Result/vocab.txt',
 '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/Result/added_tokens.json',
 '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/Result/tokenizer.json')